# VertexAI Manged Services for ADK Agents

## Get Environment Variables

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()
PROJECT = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]

print(f"PROJECT: {PROJECT}")
print(f"LOCATION: {LOCATION}")

## Initialize the VertexAI Client

In [ ]:
import vertexai

client = vertexai.Client(
    project=PROJECT,
    location=LOCATION,
)  # type: ignore

## Create an Agent Engine instance with a Memory Bank config
refs:
- https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/set-up#setup-vertex-sdk-client
- https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/agents/cloud_run/agents_with_memory/get_started_with_memory_for_adk_in_cloud_run.ipynb

In [ ]:
display_name = "[doug] Remote Session and Memory Engine"
agent_engine = client.agent_engines.create(
    config={
        "display_name": display_name,
        "context_spec": {
            "memory_bank_config": {
                "generation_config": {
                    "model": f"projects/{PROJECT}/locations/{LOCATION}/publishers/google/models/gemini-2.5-flash"  # noqa: E501
                }
            }
        },
    }
)

In [ ]:
agent_engine.__dict__

In [ ]:
agent_engine.api_resource.name

In [ ]:
agent_engine.api_resource.display_name

### Use the Agent Engine instance as a Session and Memory Service for an ADK app

Add the `agent_engine.api_resource.name` in the `session_service_uri` and `memory_service_uri` args to the `get_fast_api_app` function

(format: "projects/01234567890/locations/your-gcp-region/reasoningEngines/1234567890123456789")

```py
agent_engine_uri = f"agentengine://{agent_engine.api_resource.name}"
app = get_fast_api_app(
    agents_dir=AGENT_DIR,
    session_service_uri=agent_engine_uri,
    artifact_service_uri=artifact_service_uri,
    memory_service_uri=agent_engine_uri,
    allow_origins=allow_origins,
    web=serve_web_interface,
)
```

## Get an Agent Engine instance

In [ ]:
engine = "projects/12345678901/locations/us-central1/reasoningEngines/1234567890123456789"  # noqa: E501
retrieved_agent = client.agent_engines.get(name=engine)
retrieved_agent.api_resource.display_name

### Add a memory

In [ ]:
session_name = "projects/12345678901/locations/us-central1/reasoningEngines/1234567890123456789/sessions/123456789012345678"  # noqa: E501
client.agent_engines.memories.generate(
    name=retrieved_agent.api_resource.name,
    vertex_session_source={
        # For example, projects/.../locations/.../reasoningEngines/.../sessions/...
        "session": f"{session_name}"
    },
    # Optional when using Agent Engine Sessions. Defaults to {"user_id": session.user_id}.
    #   scope=SCOPE,
    config={"wait_for_completion": True},
)

## Delete an Agent Engine Instance

In [ ]:
# Force deletion if the engine contains memories and sessions
retrieved_agent.delete(force=True)